In [1]:
import os
os.chdir("../")
%pwd

'/home/successpear/Documents/Learning/Projects/Cat-Dog-Generation-MLFlow-DVC'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [9]:
import os
import shutil
import zipfile
import kagglehub, gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    
    def download_from_kaggle(self):
        """
        Download, extract and move to appropriate folder
        """
        # Download latest version
        dataset_url = self.config.source_URL
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        path = kagglehub.dataset_download(dataset_url)
        logger.info(f"Downloading data from {dataset_url} into file {path}")

        if not os.path.exists(unzip_path):
            shutil.copytree(os.path.join(path, "PetImages"), unzip_path)
            logger.info(f"Move dataset from cache {path} to {unzip_path}")
        else:
            logger.info(f"{unzip_path} already exist")


In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_from_kaggle()
except Exception as e:
    raise e

[2025-06-15 14:59:43,540: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-15 14:59:43,542: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-15 14:59:43,544: INFO: common: created directory at: artifacts]
[2025-06-15 14:59:43,545: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-15 14:59:44,156: INFO: 1188616259: Downloading data from shaunthesheep/microsoft-catsvsdogs-dataset into file /home/successpear/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1]
[2025-06-15 14:59:44,157: INFO: 1188616259: artifacts/data_ingestion already exist]
